In [1]:
import sys
import os
from typing import Dict, Tuple
from typing import List
from src.utils import default_data_path, config_to_env, check_config
from src.task import LaMPTask
from src.models import (
    feed_prompt_to_lm,
    feed_prompts_to_lm,
    OpenAIModel,
    task_2_parse_response,
    DistilBERTModel,
    BERTSERINIModel,
    MiniLM,
)

In [2]:
config_to_env("OPENAI_API_KEY")

In [3]:
task_header = "LaMP_2"
store_folder = os.path.join("src", "data", task_header)

OpenAI_Subscriber = lambda id, prompt, api_key: feed_prompt_to_lm(
    model=OpenAIModel(),
    id=id,
    prompt=prompt,
    api_key=api_key,
    log_path=os.path.join(store_folder, "OpenAI.txt"),
    callback=task_2_parse_response,
)
DistilBERT_Subscriber = lambda id, prompt, api_key: feed_prompt_to_lm(
    model=DistilBERTModel(task_name="LaMP_2"),
    id=id,
    prompt=prompt,
    api_key=api_key,
    log_path=os.path.join(store_folder, "DistilBERTModel.txt"),
    callback=task_2_parse_response,
)
BERTSERINI_Subscriber = lambda id, prompt, api_key: feed_prompt_to_lm(
    model=BERTSERINIModel(task_name="LaMP_2"),
    id=id,
    prompt=prompt,
    api_key=api_key,
    log_path=os.path.join(store_folder, "BERTSERINI.txt"),
    callback=task_2_parse_response,
)
MiniLM_Subscriber = lambda id, prompt, api_key: feed_prompt_to_lm(
    model=MiniLM(task_name="LaMP_2"),
    id=id,
    prompt=prompt,
    api_key=api_key,
    log_path=os.path.join(store_folder, "MiniLM.txt"),
    callback=task_2_parse_response,
)

In [4]:
store_dir = os.path.join("src", "data", task_header)
os.makedirs(store_dir, exist_ok=True)
subscribers_full = {
    "BERTSERINI": BERTSERINI_Subscriber,
    "DistilBERT": DistilBERT_Subscriber,
    "MiniLM": MiniLM_Subscriber,
    "OpenAI": OpenAI_Subscriber,
}
preds_save_path_full = {
    "BERTSERINI": os.path.join(
        store_dir,
        f"{task_header}_train_preds_BERTSERINI_with_keyword_{{file_ending}}.json",
    ),
    "DistilBERT": os.path.join(
        store_dir,
        f"{task_header}_train_preds_DistilBERT_with_keyword_{{file_ending}}.json",
    ),
    "MiniLM": os.path.join(
        store_dir,
        f"{task_header}_train_preds_MiniLM_with_keyword_{{file_ending}}.json",
    ),
    "OpenAI": os.path.join(
        store_dir, f"{task_header}_train_preds_OpenAI_with_keyword_{{file_ending}}.json"
    ),
}

task_header = "LaMP_2"
entry_per_category = 16
dataset_question_path = os.path.join("src", "data", "LaMP_2_train_questions.json")

In [5]:
def task_2_with_keywords_query(
    with_keyword_params, subscriber_namelist: List[str], debug: bool = False
):
    worker_count = 8 if not debug else 1
    api_keys = (
        [
            check_config("HUGGING_FACE_KEY_1"),
            check_config("HUGGING_FACE_KEY_2"),
            check_config("HUGGING_FACE_KEY_3"),
            check_config("HUGGING_FACE_KEY_4"),
            check_config("HUGGING_FACE_KEY_1"),
            check_config("HUGGING_FACE_KEY_2"),
            check_config("HUGGING_FACE_KEY_3"),
            check_config("HUGGING_FACE_KEY_4"),
        ]
        if not debug
        else None
    )

    ready_model = dict()

    for text_rank_top_k_keywords, category_top_k_keywords in with_keyword_params:
        subscribers = dict()
        preds_save_path = dict()
        file_ending = (
            f"{entry_per_category}_{text_rank_top_k_keywords}_{category_top_k_keywords}"
        )

        question_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_prompts_questions_with_keyword_{file_ending}.json",
        )
        output_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_outputs_selected_with_keyword_{file_ending}.json",
        )

        for subscriber_name in subscriber_namelist:
            subscribers[subscriber_name] = subscribers_full[subscriber_name]
            preds_save_path[subscriber_name] = preds_save_path_full[
                subscriber_name
            ].format(file_ending=file_ending)

        curr_task = LaMPTask(
            task_question_file=dataset_question_path,
            task_output_file=output_store_path,
            subscribers=subscribers,
            worker_count=worker_count,
            prompt_save_path=question_store_path,
            preds_save_path=preds_save_path,
            keyword_extraction=True,
        )

        ready_model[(text_rank_top_k_keywords, category_top_k_keywords)] = curr_task
        curr_task.load_prompts(question_store_path)
        curr_task.subscribe(
            skip_eval=True,
            api_keys=api_keys,
        )


def task_2_with_keywords_eval(
    with_keyword_params,
    subscriber_namelist: List[str],
    ready_models: Dict[Tuple[int, int], LaMPTask] = None,
):
    if ready_models is not None:
        for (
            text_rank_top_k_keywords,
            category_top_k_keywords,
        ), curr_task in ready_models.items():
            curr_task.evaluate()
        return

    store_dir = os.path.join("src", "data", task_header)

    evaluated_result = dict()

    for text_rank_top_k_keywords, category_top_k_keywords in with_keyword_params:
        subscribers = dict()
        preds_save_path = dict()
        file_ending = (
            f"{entry_per_category}_{text_rank_top_k_keywords}_{category_top_k_keywords}"
        )

        for subscriber_name in subscriber_namelist:
            subscribers[subscriber_name] = subscribers_full[subscriber_name]
            preds_save_path[subscriber_name] = preds_save_path_full[
                subscriber_name
            ].format(file_ending=file_ending)

        output_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_outputs_selected_with_keyword_{file_ending}.json",
        )
        curr_task = LaMPTask(
            task_question_file=dataset_question_path,
            task_output_file=output_store_path,
            subscribers=subscribers,
        )
        curr_task.evaluate(preds_save_name=preds_save_path)
        print(curr_task.score)
        evaluated_result[
            f"{text_rank_top_k_keywords} {category_top_k_keywords}"
        ] = curr_task.score
    return evaluated_result

In [6]:
with_keyword_params = [(5, 15), (10, 30)]
debug = False

In [7]:
with_keyword_params = [(5, 15)]
# with_keyword_params = [(10, 30)]
subscribers = [
    "BERTSERINI",
    # "DistilBERT",
    # "MiniLM",
    # "OpenAI"
]

# debug=True

In [9]:
ready_models = task_2_with_keywords_query(with_keyword_params, subscribers, debug=debug)

Output()

['10298', '10299', '1031', '10122', '10131', '10383', '10204', '10206', '10289', '10160', '10243', '10449', 
'10518', '10646', '10551', '10290', '10469', '10726', '10511', '10193', '10167', '10401', '10736', '10392', 
'10405', '10406', '10409', '10410', '10417', '10682', '10652', '10259', '101050', '101129', '10327', '101043', 
'101005', '101333', '101023', '101266', '101009', '101267', '101546', '101191', '101328', '101662', '101698', 
'101711', '101699', '101512', '101656', '101701', '101709', '101716', '10831', '10908', '101742', '10863', 
'101743', '10837', '101744', '102121', '101721', '102046', '102185', '101727', '102208', '101728', '10913', 
'102215', '102240', '10939', '10940', '10946', '10953', '10944', '102409', '102189', '102713', '102193', '102751', 
'102624', '102237', '102498', '101732', '102642', '102585', '102508', '102736', '102888', '102643', '102804', 
'102058', '102560', '102923', '101826', '101798', '102902', '101775', '101824', '102922', '102423', '102858', 
'102838', '102856', '102788', '102966', '102999', '102808', '101920', '103348', '102327', '101973', '103030', 
'103346', '103342', '103019', '103647', '103642', '103645', '103160', '103169', '103186', '103528', '103629', 
'103226', '103260', '103211', '103283', '103217', '103201', '103281', '103259', '103502', '104097', '103710', 
'103287', '104226', '103300', '104101', '104021', '104244', '104249', '104045', '104289', '104159', '104227', 
'104036', '104263', '104062', '104617', '104616', '104618', '104167', '104633', '104659', '104621', '104619', 
'104764', '104174', '104779', '104769', '104729', '104735', '104732', '104783', '104825', '104802', '104846', 
'103915', '104805', '103830', '104923', '105098', '104899', '103847', '105080', '105083', '104943', '105176', 
'105107', '105129', '104957', '105179', '105123']

overloaded

overloaded

overloaded

overloaded

overloaded

overloaded

overloaded

overloaded

In [13]:
evaluate_results = task_2_with_keywords_eval(with_keyword_params, subscribers)

{'MiniLM': {'accuracy': 0.25862068965517243, 'f1': 0.23614605134499464}}


In [14]:
def task_2_without_keywords_query(
    without_keyword_params, subscriber_namelist: List[str], debug: bool = False
):
    worker_count = 8 if not debug else 1
    api_keys = (
        [
            check_config("HUGGING_FACE_KEY_1"),
            check_config("HUGGING_FACE_KEY_2"),
            check_config("HUGGING_FACE_KEY_3"),
            check_config("HUGGING_FACE_KEY_4"),
            check_config("HUGGING_FACE_KEY_1"),
            check_config("HUGGING_FACE_KEY_2"),
            check_config("HUGGING_FACE_KEY_3"),
            check_config("HUGGING_FACE_KEY_4"),
        ]
        if not debug
        else None
    )

    ready_model = dict()

    for bm25_top_k in without_keyword_params:
        subscribers = dict()
        preds_save_path = dict()
        file_ending = f"{entry_per_category}_{bm25_top_k}"

        question_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_prompts_questions_without_keyword_{file_ending}.json",
        )
        output_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_outputs_selected_without_keyword_{file_ending}.json",
        )

        for subscriber_name in subscriber_namelist:
            subscribers[subscriber_name] = subscribers_full[subscriber_name]
            preds_save_path[subscriber_name] = preds_save_path_full[
                subscriber_name
            ].format(file_ending=file_ending)

        curr_task = LaMPTask(
            task_question_file=dataset_question_path,
            task_output_file=output_store_path,
            subscribers=subscribers,
            worker_count=worker_count,
            prompt_save_path=question_store_path,
            preds_save_path=preds_save_path,
            keyword_extraction=False,
        )

        ready_model[bm25_top_k] = curr_task
        curr_task.load_prompts(question_store_path)
        curr_task.subscribe(
            skip_eval=True,
            api_keys=api_keys,
        )


def task_2_without_keywords_eval(
    without_keyword_params,
    subscriber_namelist: List[str],
    ready_models: Dict[Tuple[int, int], LaMPTask] = None,
):
    if ready_models is not None:
        for bm25_top_k, curr_task in ready_models.items():
            curr_task.evaluate()
        return

    evaluated_result = dict()

    for bm25_top_k in without_keyword_params:
        subscribers = dict()
        preds_save_path = dict()
        file_ending = f"{entry_per_category}_{bm25_top_k}"
        output_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_outputs_selected_without_keyword_{file_ending}.json",
        )

        for subscriber_name in subscriber_namelist:
            subscribers[subscriber_name] = subscribers_full[subscriber_name]
            preds_save_path[subscriber_name] = preds_save_path_full[
                subscriber_name
            ].format(file_ending=file_ending)

        curr_task = LaMPTask(
            task_question_file=dataset_question_path,
            task_output_file=output_store_path,
            subscribers=subscribers,
        )
        curr_task.evaluate(preds_save_name=preds_save_path)
        print(curr_task.score)
        evaluated_result[f"{bm25_top_k}"] = curr_task.score
    return evaluated_result

In [ ]:
preds_save_path_full = {
    "BERTSERINI": os.path.join(
        store_dir,
        f"{task_header}_train_preds_BERTSERINI_without_keyword_{{file_ending}}.json",
    ),
    "DistilBERT": os.path.join(
        store_dir,
        f"{task_header}_train_preds_DistilBERT_without_keyword_{{file_ending}}.json",
    ),
    "MiniLM": os.path.join(
        store_dir,
        f"{task_header}_train_preds_MiniLM_without_keyword_{{file_ending}}.json",
    ),
    "OpenAI": os.path.join(
        store_dir,
        f"{task_header}_train_preds_OpenAI_without_keyword_{{file_ending}}.json",
    ),
}

In [15]:
debug = False
without_keyword_params = [2, 4]
# without_keyword_params = [4]
subscribers = [
    # "BERTSERINI",
    # "DistilBERT",
    "MiniLM",
    # "OpenAI"
]

In [18]:
ready_models_no_keywords = task_2_without_keywords_query(
    without_keyword_params, subscribers, debug=debug
)

Output()

['10160', '1018', '10458', '10411', '10368', '10118', '10136', '10427', '101', '10417', '10435', '10560', '10581', 
'10561', '102', '1030', '10128', '10155', '10185', '101001', '10596', '10202', '101002', '10567', '10530', 
'101028', '10798', '101007', '101031', '10831', '10959', '101049', '101172', '10772', '10897', '10850', '10826', 
'101330', '10747', '101347', '10652', '101331', '10946', '101507', '101191', '10822', '101224', '101296', '10824', 
'101662', '101151', '101701', '101721', '101724', '101218', '101696', '101438', '101699', '101478', '101490', 
'101727', '101483', '101208', '101723', '101705', '101651', '102046', '101747', '101733', '102201', '102141', 
'102053', '102033', '101688', '102172', '102126', '102220', '102130', '102232', '101773', '102483', '102233', 
'101203', '102239', '10937', '101800', '102023', '102407', '102282', '102589', '102374', '102281', '101834', 
'102824', '102717', '102496', '102514', '102098', '102647', '102665', '102810', '102888', '102634', '102976', 
'102809', '102839', '101600', '102567', '102224', '102838', '103052', '102694', '102741', '103423', '102915', 
'103350', '102858', '102662', '103256', '103392', '103178', '103518', '103242', '103482', '103450', '103293', 
'103600', '103504', '103616', '103558', '103595', '103586', '103641', '103167', '103621', '103326', '102969', 
'103557', '103281', '102736', '103300', '104037', '104069', '103592', '103711', '103527', '103511', '104270', 
'103321', '104281', '104087', '104050', '104275', '104196', '104671', '104629', '104654', '103980', '103741', 
'104110', '104771', '104743', '104633', '104780', '103686', '104747', '103880', '104637', '103708', '104727', 
'104367', '104381', '104304', '104479', '103940', '104814', '104457', '104851', '105080', '104873', '104887', 
'104833', '105099', '105333', '105299', '105136', '105102', '105297', '105326', '105148', '105412', '105031', 
'104981', '105284', '104909', '104830', '105606', '105245', '105100', '105308', '104863', '105730', '105623', 
'105743', '105544', '105740', '105429', '105782', '105765', '105754', '105636', '105674', '105523', '105771', 
'104991', '105533', '105769', '105538', '105763', '105744', '105722', '105899', '105648', '105834', '105814', 
'105821', '102045']

Output()

['10295', '10139', '10134', '1013', '10166', '10441', '10388', '10364', '10419', '10169', '10406', '10164', '1024',
'10638', '1051', '10583', '10153', '10729', '10560', '10571', '10502', '10551', '10549', '10699', '10973', 
'101014', '1094', '10243', '101004', '101013', '10387', '101039', '101070', '10715', '101019', '10969', '101026', 
'101046', '101140', '101302', '101275', '101257', '101246', '101320', '101049', '10789', '101287', '10234', 
'10168', '10946', '10327', '10205', '10297', '10482', '104', '101345', '101164', '101337', '101340', '101698', 
'10752', '101727', '101720', '101351', '10808', '101701', '101705', '101600', '101355', '101719', '101854', 
'101735', '101193', '10213', '101750', '101619', '101279', '101653', '101728', '102140', '10824', '102238', 
'101798', '102130', '102409', '102380', '102096', '102233', '102587', '102064', '102523', '102041', '102821', 
'102181', '102054', '102368', '102813', '102824', '101983', '102347', '102664', '101760', '102222', '102638', 
'102848', '102736', '101919', '102796', '103012', '102746', '102822', '103346', '102918', '102922', '103400', 
'102756', '103205', '103235', '102707', '103165', '102973', '103283', '103626', '103576', '103641', '102958', 
'103298', '103470', '103558', '103519', '103485', '103512', '103312', '103311', '103352', '103162', '104061', 
'104052', '104031', '103690', '104276', '103698', '103267', '104050', '103965', '104621', '104650', '104244', 
'104637', '104623', '104196', '103675', '104292', '104750', '103770', '104775', '104783', '104758', '104048', 
'104117', '104087', '104751', '104112', '104786', '104771', '103812', '104632', '104939', '103993', '104013', 
'104385', '103745', '105020', '104850', '104877', '104796', '104992', '104987', '105073', '105070', '104791', 
'104922', '105105', '104972', '105089', '105083', '105359', '104874', '105085', '105084', '104844', '105133', 
'105015', '104890', '105107', '105575', '105054', '105562', '105118', '105627', '105623', '105625', '105725', 
'105417', '105753', '105779', '105768', '105218', '105693', '105275', '105728', '105148', '105782', '105850', 
'105566', '105761', '105061', '105882', '105399', '105507', '105230', '105906', '105524', '105884', '105879', 
'105911', '105847', '105908', '105487', '105862', '105872', '105855']

In [19]:
evaluate_results_no_keywords = task_2_without_keywords_eval(
    without_keyword_params, subscribers
)

{'MiniLM': {'accuracy': 0.2511013215859031, 'f1': 0.23674196174196174}}
{'MiniLM': {'accuracy': 0.3276595744680851, 'f1': 0.29632111746810086}}
